# Transfer FGSM attack generator - Surrogate Model: Resnext + LSTM (sequence based)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
# Imports
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm
from moviepy.editor import ImageSequenceClip

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

In [ ]:
# Model Definition
class Model(nn.Module):
    def __init__(self, num_classes, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(hidden_dim if bidirectional else latent_dim, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = x.view(b * t, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(b, t, 2048)
        x_lstm, _ = self.lstm(x)
        return fmap, self.dp(self.linear1(x_lstm[:, -1, :]))


# Load model from given checkpoint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT = "/content/drive/MyDrive/Models/model_87_acc_20_frames_final_data.pt"

_model = Model(2).to(device)
_model.load_state_dict(torch.load(CHECKPOINT, map_location=device))
_model.eval()

In [ ]:
# Image Preprocessing
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

def preprocess_frame(frame_bgr):
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    return _transform(frame_rgb)

In [ ]:
# Helper functions for Sequence level attack
def fgsm_attack_sequence(frames_bgr, true_label, epsilon=0.01):
    _model.train()

    tensors = [preprocess_frame(f) for f in frames_bgr]
    x = torch.stack(tensors).unsqueeze(0).to(device)  # (1, T, 3,112,112)
    x.requires_grad = True

    _, logits = _model(x)
    target = torch.tensor([true_label]).to(device) # Use true label --> untargetted

    loss = F.cross_entropy(logits, target)
    loss.backward()

    grad = x.grad.data
    x_adv = x + epsilon * grad.sign()
    x_adv = torch.clamp(x_adv, -3, 3)

    _model.eval()

    adv_np = x_adv.detach().cpu().numpy()[0]

    adv_frames = []
    for i in range(adv_np.shape[0]):
        f = adv_np[i].transpose(1,2,0)
        f = f * np.array(std) + np.array(mean)
        f = np.clip(f * 255, 0, 255).astype(np.uint8)
        adv_frames.append(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))

    return adv_frames



def fgsm_attack_video(input_path, output_path, true_label, epsilon=0.01, max_frames=70):

    cap = cv2.VideoCapture(input_path)
    frames = []
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret or count >= max_frames:
            break
        frames.append(frame)
        count += 1

    cap.release()

    if len(frames) == 0:
        print("no frames extracted")
        return

    adv_frames = fgsm_attack_sequence(frames, true_label, epsilon)

    rgb_frames = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in adv_frames]
    clip = ImageSequenceClip(rgb_frames, fps=25)
    clip = ImageSequenceClip(frames, fps=25)
    clip.write_videofile(
        output_path,
        codec="libx264",
        audio=False,
        verbose=False,
        logger=None
    )


In [ ]:
# src and dst (change to the folder where your deepfakes are and where you want to save it)
src_dir = "/content/drive/MyDrive/faceforensics++/manipulated_sequences/DeepFakeDetection/c40/videos/"
output_dir = "/content/drive/MyDrive/faceforensics++/Adversarial_attacked_sequences/TransferAttacks/FGSM/DeepfakeDetector/ResNext_LTSM_87_acc_20_sequence/DeepfakeDetection/"

In [ ]:
# Wrapper function for fgsm attack video
def fgsm_attack(video_path, output_path, epsilon, true_label=0, max_frames=70):
    return fgsm_attack_video(
        input_path=video_path,
        output_path=output_path,
        true_label=true_label,
        epsilon=epsilon,
        max_frames=max_frames
    )


def FGSM(epsilon, true_label=0):
    outdir = os.path.join(output_dir, f"Epsilon{epsilon}")
    os.makedirs(outdir, exist_ok=True)

    for video_path in tqdm(glob.glob(os.path.join(src_dir, "*.mp4"))):
        fname = os.path.basename(video_path)
        save_path = os.path.join(outdir, fname)

        if os.path.exists(save_path):
            continue

        fgsm_attack(
            video_path=video_path,
            output_path=save_path,
            epsilon=epsilon,
            true_label=true_label
        )

In [ ]:
FGSM(0.01)

  5%|▍         | 152/3068 [03:21<1:04:18,  1.32s/it]


KeyboardInterrupt: 